# Imports

In [1]:
import json
import pandas as pd

In [8]:
# digital twin package
from  terminal_optimization.core import vessel_properties_mixin, cyclic_properties_mixin, continuous_properties_mixin

# Create necessary classes

## general classes

In [2]:
# create vessel class
class vessel(vessel_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [3]:
# create unloader class
class cyclic_unloader(cyclic_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [4]:
# create unloader class
class continuous_unloader(continuous_properties_mixin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [ ]:
# create quay class (not finalised yet)
class quay():
    def __init__(self):
        
    def max_unloader_per_berth(t):
        if n_calls_panamax(t) != 0: 
            return 3
        if n_calls_panamax(t) == 0:
            return 2

## trends and splits

In [130]:
# create bulk commodities class
class bulk_commodities:
    def __init__(self, fname):
        self.trend_forecasts = pd.read_excel(fname, 'TF')
        self.years = self.trend_forecasts["Year"]
        self.start_year = years[0]
        self.stop_year  = years[len(years)-1]
        self.n_years    = len(years)
    def get_forecast(self,commodity_name):
        return self.trend_forecasts[commodity_name]

In [247]:
# create bulk commodities class
class bulk_modal_splits:
    def __init__(self, fname):
        self.split_forecasts = pd.read_excel(fname, 'Vessel distribution')
        self.years = self.split_forecasts["Year"]
        self.start_year = years[0]
        self.stop_year  = years[len(years)-1]
        self.n_years    = len(years)
        
    def get_forecast(self,commodity_name):
        # the hard coded columns 1:4, 4:7 and 7:10 indicate that the datamodel is not yet ideal
        if commodity_name == "Maize":
            return self.split_forecasts[modal_splits.split_forecasts.columns[1:4]]
        elif commodity_name == "Soybeans":
            return self.split_forecasts[modal_splits.split_forecasts.columns[4:7]]
        elif commodity_name == "Wheat":
            return self.split_forecasts[modal_splits.split_forecasts.columns[7:10]]


In [ ]:
# not yet looked at, only moved to the source code section
# should be possible to simplify according to the suggestion below
# calling could then just be one simple line
# whealth accumulation factor
class WACC:
    def __init__(self):
        pass
    
    def get_WACC(self, single_parameters, n_years, owner)
        # The nominal WACC of both entities (Port Authority and Terminal Operator)
        real_WACC_PA   = single_parameters["PA_WACC_real"]
        real_WACC_TO   = single_parameters["TO_WACC_real"]
        inflation_rate = single_parameters["InflationRate"]

        PA_WACC        = np.zeros (n_years)
        TO_WACC        = np.zeros (n_years)
        for i in range (n_years):
            PA_WACC[i] = 1 / ((1 + real_WACC_PA + inflation_rate)**(i))
            TO_WACC[i] = 1 / ((1 + real_WACC_TO + inflation_rate)**(i))

        PA_WACC = PA_WACC.reshape((-1, 1))
        TO_WACC = TO_WACC.reshape((-1, 1))
    
        if owner == 'Port authority':
            return PA_WACC
        elif owner == 'Terminal operator':
            return TO_WACC    


In [ ]:
# not yet looked at, only moved to the source code section
# Euro adjustment (source: ECB)
def adjustment_factor_euro(year):
    inflation_data = [1.20, 1.54, 0.24, 0.01, 0.44, 1.35, 2.50, 2.71, 1.62, 0.29, 3.29, 2.13, 2.19, 2.18, 2.14, 2.08, 2.27, 2.36, 2.11, 1.08, 1.09, 1.58]
    data = np.zeros(2018 - year + 1)
    for i in range (2018 - year + 1):
        data[i] = 1+(inflation_data[i]/100)
    return np.prod(data)


In [ ]:
# not yet looked at, only moved to the source code section
# Dollar adjustment (source: IMF)
def adjustment_factor_dollar(year):
    inflation_data = [2.34, 1.78, 0.45, 0.06, 0.45, 1.35, 2.45, 2.74, 1.62, 0.42, 3.09, 2.28, 2.19, 2.30, 2.22, 2.07, 2.29, 2.30, 2.21, 1.23, 1.13, 1.58]
    data = np.zeros(2018 - year + 1)
    for i in range (2018 - year + 1):
        data[i] = 1+(inflation_data[i]/100)
    return np.prod(data)


In [ ]:
# not yet looked at, only moved to the source code section
# Euro to dollar exchange (source: ECB and averaged over 2018)
def Euro_to_dollars(euros):
    return euros * 1.1924

# Simulation

## Prepare simulation inputs

In [86]:
fname = 'Excel_input.xlsx'

## vessels

In [8]:
# here I created a JSON to prepare the inputs. 
# Since this is presumably not very variable input it can be standardized like this.

# create input
handysize_data = {"vessel_type": 'Handysize', "call_size": 35000, 
                  "LOA": 130, "draft": 10, "beam": 24, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 600} 
handymax_data = {"vessel_type": 'Handymax', "call_size": 50000, 
                  "LOA": 180, "draft": 11.5, "beam": 28, "max_cranes": 2, 
                  "all_turn_time": 24, "mooring_time": 3, "demurrage": 750} 
panamax_data = {"vessel_type": 'Panamax', "call_size": 65000, 
                  "LOA": 220, "draft": 13, "beam": 32.2, "max_cranes": 3, 
                  "all_turn_time": 36, "mooring_time": 4, "demurrage": 730} 

# create objects
handysize = vessel(**handysize_data)
handymax = vessel(**handymax_data)
panamax = vessel(**panamax_data)

# print object dicts
# print(handysize.__dict__)
# print(handymax.__dict__)
# print(panamax.__dict__)

## cranes

In [248]:
# here I created a JSON to prepare the inputs. 
# Since this is presumably not very variable input it can be standardized like this.

# create input
gantry_crane_data   = {"unloader_type": 'Gantry crane', "lifting_capacity": 50, 
                       "hourly_cycles": 60, "eff_fact": 0.6} 
harbour_crane_data  = {"unloader_type": 'Harbour crane', "lifting_capacity": 40, 
                       "hourly_cycles": 40, "eff_fact": 0.6} 
mobile_crane_data   = {"unloader_type": 'Mobile crane', "lifting_capacity": 40, 
                       "hourly_cycles": 30, "eff_fact": 0.5} 
screw_unloader_data = {"unloader_type": 'Screw unloader', "peak_capacity": 1400, 
                       "eff_fact": 0.55} 

# create objects
gantry_crane    = cyclic_unloader(**gantry_crane_data)       
harbour_crane   = cyclic_unloader(**harbour_crane_data)       
mobile_crane    = cyclic_unloader(**mobile_crane_data)
screw_unloader  = continuous_unloader(**screw_unloader_data)

# print object dicts
# print(gantry_crane.__dict__)
# print(harbour_crane.__dict__)
# print(mobile_crane.__dict__)
# print(screw_unloader.__dict__)

## forecasts

In [249]:
# get commodity forecast from XLS
commodities = bulk_commodities(fname)

# extract specifics for Maize
tf_maize = commodities.get_forecast("Maize")

In [253]:
# get modal split forecast from XLS
modal_splits = bulk_modal_splits(fname)

# extract specifics for Maize
tf_mod_split = modal_splits.get_forecast("Maize")

## Single parameters


## Perform calculations

### Calculation settings

In [ ]:
start_year = commodities.start_year
stop_year  = commodities.stop_year
n_years    = commodities.n_years

In [ ]:
single_parameters = {
    "AnnualOperationalHours": 8760,
    "LengthImportConveyor": 500,
    "LengthExportConveyor": 800,
    "PA_WACC_real": 0.05,
    "TO_WACC_real": 0.07,
    "InflationRate": 0.03,
    "DemurrageEscalation": 0.03,
    "HandlingFeeEscalation": 0.03,
    "CapexEscalation": 0.03,
    "LabourEscalation": 0.05,
    "MaintenanceEscalation": 0.05,
    "Revenues": 'Terminal operator',
    "DemurrageCosts": 'Terminal operator',
    "QuayWall": 'Port authority',
    "Equipment": 'Terminal operator',
    "UnloadersDepreciation": 0.03,
    "UnloadersSellingFee": 0.1,
    "QuayDepreciation": 0.02,
    "QuaySellingFee": 0.1}


# Postprocess results